In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob
import PIL
import gc
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.models import Sequential

In [2]:
!conda install ../input/how-to-use-pyvips-offline/*.tar.bz2



######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
###########################

In [3]:
import pyvips

In [4]:
Image.MAX_IMAGE_PIXELS = None

In [5]:
train_data = pd.read_csv("../input/mayo-clinic-strip-ai/train.csv")
test_data = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
display(train_data)

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE
...,...,...,...,...,...
749,fe9645_0,3,fe9645,0,CE
750,fe9bec_0,4,fe9bec,0,LAA
751,ff14e0_0,6,ff14e0,0,CE
752,ffec5c_0,7,ffec5c,0,LAA


In [6]:
train_df = pd.DataFrame(columns=['Path','image_id'])
train_df['Path'] = glob.glob("../input/mayo-convert-tif-to-jpg/train/*")
train_df['image_id'] = train_df['Path'].apply(lambda x: x.split('/')[-1].replace('.jpg',""))
train_df


,Path,image_id
0,../input/mayo-convert-tif-to-jpg/train/c9d53d_...,c9d53d_0
1,../input/mayo-convert-tif-to-jpg/train/0b7cc8_...,0b7cc8_0
2,../input/mayo-convert-tif-to-jpg/train/ea2aad_...,ea2aad_0
3,../input/mayo-convert-tif-to-jpg/train/8cf32b_...,8cf32b_0
4,../input/mayo-convert-tif-to-jpg/train/46def4_...,46def4_0
...,...,...
749,../input/mayo-convert-tif-to-jpg/train/04439c_...,04439c_0
750,../input/mayo-convert-tif-to-jpg/train/56d177_...,56d177_3
751,../input/mayo-convert-tif-to-jpg/train/509042_...,509042_0
752,../input/mayo-convert-tif-to-jpg/train/a26055_...,a26055_0


In [7]:
train_data = train_data.merge(train_df,on='image_id')
train_data

,image_id,center_id,patient_id,image_num,label,Path
0,006388_0,11,006388,0,CE,../input/mayo-convert-tif-to-jpg/train/006388_...
1,008e5c_0,11,008e5c,0,CE,../input/mayo-convert-tif-to-jpg/train/008e5c_...
2,00c058_0,11,00c058,0,LAA,../input/mayo-convert-tif-to-jpg/train/00c058_...
3,01adc5_0,11,01adc5,0,LAA,../input/mayo-convert-tif-to-jpg/train/01adc5_...
4,026c97_0,4,026c97,0,CE,../input/mayo-convert-tif-to-jpg/train/026c97_...
...,...,...,...,...,...,...
749,fe9645_0,3,fe9645,0,CE,../input/mayo-convert-tif-to-jpg/train/fe9645_...
750,fe9bec_0,4,fe9bec,0,LAA,../input/mayo-convert-tif-to-jpg/train/fe9bec_...
751,ff14e0_0,6,ff14e0,0,CE,../input/mayo-convert-tif-to-jpg/train/ff14e0_...
752,ffec5c_0,7,ffec5c,0,LAA,../input/mayo-convert-tif-to-jpg/train/ffec5c_...


In [8]:
#img = Image.open(train_data['Path'][0])
#plt.imshow(img)

In [9]:
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])
train_data


,image_id,center_id,patient_id,image_num,label,Path
0,006388_0,11,006388,0,0,../input/mayo-convert-tif-to-jpg/train/006388_...
1,008e5c_0,11,008e5c,0,0,../input/mayo-convert-tif-to-jpg/train/008e5c_...
2,00c058_0,11,00c058,0,1,../input/mayo-convert-tif-to-jpg/train/00c058_...
3,01adc5_0,11,01adc5,0,1,../input/mayo-convert-tif-to-jpg/train/01adc5_...
4,026c97_0,4,026c97,0,0,../input/mayo-convert-tif-to-jpg/train/026c97_...
...,...,...,...,...,...,...
749,fe9645_0,3,fe9645,0,0,../input/mayo-convert-tif-to-jpg/train/fe9645_...
750,fe9bec_0,4,fe9bec,0,1,../input/mayo-convert-tif-to-jpg/train/fe9bec_...
751,ff14e0_0,6,ff14e0,0,0,../input/mayo-convert-tif-to-jpg/train/ff14e0_...
752,ffec5c_0,7,ffec5c,0,1,../input/mayo-convert-tif-to-jpg/train/ffec5c_...


(Try reducing number of CE data in training )


In [10]:
y = train_data["label"]
train, test = train_test_split(train_data, test_size=0.25, stratify = y)

train = train.reset_index(drop=True)
train

,image_id,center_id,patient_id,image_num,label,Path
0,41b23c_0,3,41b23c,0,1,../input/mayo-convert-tif-to-jpg/train/41b23c_...
1,862501_1,7,862501,1,1,../input/mayo-convert-tif-to-jpg/train/862501_...
2,dba56f_0,4,dba56f,0,1,../input/mayo-convert-tif-to-jpg/train/dba56f_...
3,62dd90_0,9,62dd90,0,0,../input/mayo-convert-tif-to-jpg/train/62dd90_...
4,516e18_1,10,516e18,1,0,../input/mayo-convert-tif-to-jpg/train/516e18_...
...,...,...,...,...,...,...
560,45e3e0_0,11,45e3e0,0,0,../input/mayo-convert-tif-to-jpg/train/45e3e0_...
561,f322da_0,11,f322da,0,0,../input/mayo-convert-tif-to-jpg/train/f322da_...
562,9fa03a_0,2,9fa03a,0,0,../input/mayo-convert-tif-to-jpg/train/9fa03a_...
563,f9fc6b_1,11,f9fc6b,1,1,../input/mayo-convert-tif-to-jpg/train/f9fc6b_...


In [11]:
train['label'].value_counts()

0    410
1    155
Name: label, dtype: int64

In [12]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, df, batch_size=8, n_classes=2, shuffle=False,to_fit=True):
        'Initialization'
        self.df = df
        self.to_fit = to_fit
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.df))
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size: (index+1)*self.batch_size]
        X = np.empty((self.batch_size,512,512,3),dtype=np.float32)
        y = np.empty((self.batch_size,1),dtype=np.float32)
        
        if self.to_fit == True:
            
            for i, img_path in enumerate(self.df['Path'].iloc[indexes]):
                img = self.read_img(img_path)
                X[i,] = img
                y[i] = self.df['label'].iloc[indexes[i]]
            return X, y
        
        else:
            for i, img_path in enumerate(self.df['Path'].iloc[indexes]):
                img = self.read_img(img_path)
                X[i,] = img
                
            return X
        
        
    def read_img(self, path):
        image =  pyvips.Image.thumbnail(path,512,height=512,size='force')
        image = np.asarray(image)
        image = image.astype(np.float32)
        return image/255
        

In [13]:
training_generator = DataGenerator(train,shuffle = True)
val_generator = DataGenerator(test,shuffle = True)

**Loss function** taken from : 
https://www.kaggle.com/code/towhidultonmoy/implementing-custom-weighted-multi-class-log-loss¶

In [14]:

epsilon = 1e-7 
n_classes=2 #As there are two classes in the dataset
weights = np.ones(n_classes, dtype='float32')  #For this competition, you can use weights from probing the leaderboard
class_counts = train_data.groupby('label')['image_id'].count().values 
class_proportions = class_counts/np.max(class_counts)
K.set_floatx('float32') #You can also set backend to any float 

In [15]:
def weighted_mc_log_loss(y_true, y_pred):
    # Clipping the prediction value
    y_pred_clipped = K.clip(y_pred, epsilon, 1-epsilon)  
    #true labels weighted by weights and percent elements per class
    y_true_weighted = (y_true * weights)/class_proportions
    #multiply tensors element-wise and then sum
    loss_num = (y_true_weighted * K.log(y_pred_clipped))
    loss = -1*K.sum(loss_num)/K.sum(weights)
    
    return loss

In [16]:
input_shape=(512,512,3)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

2022-08-14 10:04:21.732195: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 10:04:21.862409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 10:04:21.863219: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 10:04:21.864467: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [17]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
os.makedirs('best_model')

In [19]:
callbacks=[
           tf.keras.callbacks.EarlyStopping(patience=5,monitor='loss'),
         tf.keras.callbacks.ReduceLROnPlateau( monitor='loss',factor=0.1,patience=3, mode='min',min_lr=0.00001),
        tf.keras.callbacks.ModelCheckpoint(filepath='./best_model',monitor = 'val_accuracy', save_best_only=True)
]

In [20]:
history = model.fit(training_generator,
          batch_size=8,
          epochs=5,
          validation_data = val_generator,
          shuffle = True,
          callbacks=callbacks)

2022-08-14 10:04:26.355639: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2022-08-14 10:04:30.501660: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


70/70 [==============================] - 233s 3s/step - loss: 21.9281 - accuracy: 0.9214 - val_loss: 0.6514 - val_accuracy: 1.0000


2022-08-14 10:08:19.680411: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-08-14 10:08:20.330166: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2114060288 exceeds 10% of free system memory.
2022-08-14 10:08:22.429154: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2114060288 exceeds 10% of free system memory.
2022-08-14 10:08:24.215029: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2114060288 exceeds 10% of free system memory.


Epoch 2/5
70/70 [==============================] - 218s 3s/step - loss: 0.6591 - accuracy: 0.9714 - val_loss: 0.6138 - val_accuracy: 0.8913
Epoch 3/5
70/70 [==============================] - 218s 3s/step - loss: 0.5318 - accuracy: 0.9125 - val_loss: 0.6431 - val_accuracy: 0.9837
Epoch 4/5
70/70 [==============================] - 216s 3s/step - loss: 0.3597 - accuracy: 0.8500 - val_loss: 0.9012 - val_accuracy: 0.9783
Epoch 5/5
70/70 [==============================] - 217s 3s/step - loss: 0.1550 - accuracy: 0.7661 - val_loss: 1.1656 - val_accuracy: 0.9076


In [21]:
test_data

,image_id,center_id,patient_id,image_num
0,006388_0,11,006388,0
1,008e5c_0,11,008e5c,0
2,00c058_0,11,00c058,0
3,01adc5_0,11,01adc5,0


In [22]:
test_path = glob.glob("../input/mayo-clinic-strip-ai/test/*")
test_df = pd.DataFrame(columns=['Path','image_id'])
test_df['Path'] = test_path
test_df['image_id'] = test_df['Path'].apply(lambda x: x.split('/')[-1].replace('.tif',''))
test_df

,Path,image_id
0,../input/mayo-clinic-strip-ai/test/006388_0.tif,006388_0
1,../input/mayo-clinic-strip-ai/test/00c058_0.tif,00c058_0
2,../input/mayo-clinic-strip-ai/test/008e5c_0.tif,008e5c_0
3,../input/mayo-clinic-strip-ai/test/01adc5_0.tif,01adc5_0


In [23]:
test_data = test_data.merge(test_df,on='image_id')
test_data

,image_id,center_id,patient_id,image_num,Path
0,006388_0,11,006388,0,../input/mayo-clinic-strip-ai/test/006388_0.tif
1,008e5c_0,11,008e5c,0,../input/mayo-clinic-strip-ai/test/008e5c_0.tif
2,00c058_0,11,00c058,0,../input/mayo-clinic-strip-ai/test/00c058_0.tif
3,01adc5_0,11,01adc5,0,../input/mayo-clinic-strip-ai/test/01adc5_0.tif


In [24]:
test_gen = DataGenerator(test_data,batch_size=1,to_fit=False)


In [25]:
#from keras.models import load_model
#saved_model = load_model('./best_model')

In [26]:
predictions = model.predict(test_gen, verbose=1)

4/4 [==============================] - 126s 26s/step


In [27]:
df_preds = pd.DataFrame(predictions, columns=["CE", "LAA"])
df_preds['patient_id'] = test_data['patient_id'] 
df_preds.head()

,CE,LAA,patient_id
0,1.000000,9.159411e-09,006388
1,1.000000,1.864462e-07,008e5c
2,0.001434,9.985659e-01,00c058
3,0.000134,9.998661e-01,01adc5


In [28]:
sub = {'patient_id':df_preds['patient_id'],'CE': df_preds['CE'],'LAA':df_preds['LAA']}
submission = pd.DataFrame(sub)
submission = submission.groupby("patient_id").mean().reset_index()
submission

,patient_id,CE,LAA
0,006388,1.000000,9.159411e-09
1,008e5c,1.000000,1.864462e-07
2,00c058,0.001434,9.985659e-01
3,01adc5,0.000134,9.998661e-01


In [29]:
submission[["patient_id", "CE", "LAA"]].to_csv("/kaggle/working/submission.csv", index=False)



!head submission.csv

patient_id,CE,LAA
006388,1.0,9.159411e-09
008e5c,0.99999976,1.8644619e-07
00c058,0.0014341802,0.99856585
01adc5,0.0001338474,0.9998661
